In [1]:
import pandas as pd
import re
import numpy as np
import os
import glob

In [2]:
import configparser

config = configparser.RawConfigParser()
config.read('parametros.config')

PARAMETROS = dict(config.items('Padronizacao_conversao'))
PARAMETROS_GERAIS = dict(config.items('Geral'))

In [3]:
regex = {}
regex['ATIVO']                    = "1            "
regex['ATIVO_CIRCULANTE']         = "12           "
regex['CREDITO_OPER_PS']          = "123          "
regex['PASSIVO']                  = "2            "
regex['PASSIVO_CIRCULANTE']       = "21           "
regex['PASSIVO_NAO_CIRCULANTE']   = "23           "
regex['PL']                       = "25           "
regex['RECEITA']                  = "31           "
regex['CONTRAPRESTACAO_PRE']      = "311..2       "
regex['CONTRAPRESTACAO_POS']      = "311..1       "
regex['CORRESP_CEDIDA_PRE']       = "3117.1       "
regex['CORRESP_CEDIDA_POS']       = "3117.2       "
regex['DESPESA']                  = "41           "
regex['EVENTOS_INDENIZAVEIS_PRE'] = "411..1       "
regex['EVENTOS_INDENIZAVEIS_POS'] = "411..2       "
regex['PEONA']                    = "414          "
regex['DESPESA_COM']              = "43           "
regex['DESPESA_ADM']              = "46           "



In [4]:
PASTA_DEMONSTRACOES_POR_ANO = PARAMETROS_GERAIS['pasta_raiz_dados']+PARAMETROS['pasta_arquivos_contenados']

In [5]:
lista_arquivos = [i for i in glob.glob(PASTA_DEMONSTRACOES_POR_ANO+"/*")]

In [6]:
lista_arquivos

['/opt/DADOS_ANS_RESUMIDO/todas_demonstracoes_por_ano/2010todas_demonstracoes_concatenadas.csv',
 '/opt/DADOS_ANS_RESUMIDO/todas_demonstracoes_por_ano/2019todas_demonstracoes_concatenadas.csv',
 '/opt/DADOS_ANS_RESUMIDO/todas_demonstracoes_por_ano/2013todas_demonstracoes_concatenadas.csv',
 '/opt/DADOS_ANS_RESUMIDO/todas_demonstracoes_por_ano/2018todas_demonstracoes_concatenadas.csv',
 '/opt/DADOS_ANS_RESUMIDO/todas_demonstracoes_por_ano/2011todas_demonstracoes_concatenadas.csv',
 '/opt/DADOS_ANS_RESUMIDO/todas_demonstracoes_por_ano/2012todas_demonstracoes_concatenadas.csv',
 '/opt/DADOS_ANS_RESUMIDO/todas_demonstracoes_por_ano/2014todas_demonstracoes_concatenadas.csv',
 '/opt/DADOS_ANS_RESUMIDO/todas_demonstracoes_por_ano/2015todas_demonstracoes_concatenadas.csv',
 '/opt/DADOS_ANS_RESUMIDO/todas_demonstracoes_por_ano/2016todas_demonstracoes_concatenadas.csv',
 '/opt/DADOS_ANS_RESUMIDO/todas_demonstracoes_por_ano/2017todas_demonstracoes_concatenadas.csv']

In [7]:
def ordenar_dataset(data):
    return data.sort_values(by=['ANO','SEMESTRE',"REG_ANS","CD_CONTA_CONTABIL"])

In [8]:
def ajustar_linhas(data):
    data['CD_CONTA_CONTABIL'] = data['CD_CONTA_CONTABIL'].str.ljust(width=13, fillchar=' ')
    data['VL_SALDO_FINAL'] = data['VL_SALDO_FINAL'].str.replace(pat=',', repl='.').astype(float)
    data = ordenar_dataset(data)
    return data

In [9]:
def recuperar_trimestre_anterior(item,data):
#     print("Recuperar trimestre anterior de ",item.name)
    try: 
        chave_anterior = item.name[:item.name.rfind("_")+1]
        semestre_anterior = int(item['SEMESTRE']) - 1
        if (semestre_anterior == 0):
            return None
        chave_anterior = chave_anterior+str(semestre_anterior)
        item_anterior =  data.loc[[chave_anterior]]
        if(len(item_anterior) > 0):
            item_anterior = item_anterior.squeeze()
            return item_anterior
    except:
        return None

In [10]:
def process(item,data,_nome_conta):
    valor_atual = item['VL_SALDO_FINAL']
    if (item['SEMESTRE'] != "1"):
        try:
            item_anterior = recuperar_trimestre_anterior(item,data)
            valor_atual = valor_atual - item_anterior['VL_SALDO_FINAL']
        except:
            pass
    item[_nome_conta] = str(float("{0:.2f}".format((valor_atual))))
    return item    

In [11]:
def ajustar_contas(_nome_conta, _regex_conta, data, lista_codigos):
    print(_nome_conta,_regex_conta)
    r = re.compile(_regex_conta)
    lista_match_regex = list(filter(r.match, lista_codigos)) 
    df_contas = data[data['CD_CONTA_CONTABIL'].isin(lista_match_regex)]
    df_contas.insert(loc=0, column='pos', value=np.arange(len(df_contas)))
    df_contas = df_contas.apply(lambda x : process(x,df_contas,_nome_conta),axis=1)
    df_contas.drop(columns=['pos'],inplace=True)
    return df_contas

In [12]:
if (not(os.path.exists("resultado"))):
    os.mkdir("resultado")
for _arquivo in lista_arquivos:
    data = pd.read_csv(_arquivo, dtype=str)
    print(_arquivo)
    data = data.set_index("CHAVE")
    data = ajustar_linhas(data)
    lista_codigos_conta_contabil = data['CD_CONTA_CONTABIL'].unique().tolist()    
    for _nome_conta, _regex_conta in regex.items():
        data.insert(loc=3, column=_nome_conta, value=0.0)
        df_contas = ajustar_contas(_nome_conta,_regex_conta,data,lista_codigos_conta_contabil)
        data.drop(df_contas.index,inplace=True)
        data = pd.concat([data,df_contas])
    nome_arquivo = _arquivo[_arquivo.rfind(os.path.sep)+1:]
    print(nome_arquivo)
    data = ordenar_dataset(data)
    data.to_csv("resultado"+os.path.sep+nome_arquivo)

/opt/DADOS_ANS_RESUMIDO/todas_demonstracoes_por_ano/2010todas_demonstracoes_concatenadas.csv
ATIVO 1            
ATIVO_CIRCULANTE 12           
CREDITO_OPER_PS 123          
PASSIVO 2            
PASSIVO_CIRCULANTE 21           
PASSIVO_NAO_CIRCULANTE 23           
PL 25           
RECEITA 31           
CONTRAPRESTACAO_PRE 311..2       
CONTRAPRESTACAO_POS 311..1       
CORRESP_CEDIDA_PRE 3117.1       
CORRESP_CEDIDA_POS 3117.2       
DESPESA 41           
EVENTOS_INDENIZAVEIS_PRE 411..1       
EVENTOS_INDENIZAVEIS_POS 411..2       
PEONA 414          
DESPESA_COM 43           
DESPESA_ADM 46           
2010todas_demonstracoes_concatenadas.csv
/opt/DADOS_ANS_RESUMIDO/todas_demonstracoes_por_ano/2019todas_demonstracoes_concatenadas.csv
ATIVO 1            
ATIVO_CIRCULANTE 12           
CREDITO_OPER_PS 123          
PASSIVO 2            
PASSIVO_CIRCULANTE 21           
PASSIVO_NAO_CIRCULANTE 23           
PL 25           
RECEITA 31           
CONTRAPRESTACAO_PRE 311..2       
CONTRAPRE

In [13]:
lista_arquivos = [i for i in glob.glob("resultado/*")]

In [14]:
df_demonstracoes_concatenadas = pd.concat([pd.read_csv(f,dtype=str) for f in lista_arquivos],sort=False)

In [15]:
len(df_demonstracoes_concatenadas)

7580861

In [16]:
df_demonstracoes_concatenadas.to_csv("todas_demonstracoes.csv")